<h1>Table of Contents<span class="tocSkip"></span></h1>
<div class="toc"><ul class="toc-item"><li><span><a href="#Composite-Pattern" data-toc-modified-id="Composite-Pattern-1"><span class="toc-item-num">1&nbsp;&nbsp;</span>Composite Pattern</a></span><ul class="toc-item"><li><ul class="toc-item"><li><span><a href="#A-classic-composite/noncomposite-hierarchy" data-toc-modified-id="A-classic-composite/noncomposite-hierarchy-1.0.1"><span class="toc-item-num">1.0.1&nbsp;&nbsp;</span>A classic composite/noncomposite hierarchy</a></span></li><li><span><a href="#A-single-class-for-both-composites-and-non-composites" data-toc-modified-id="A-single-class-for-both-composites-and-non-composites-1.0.2"><span class="toc-item-num">1.0.2&nbsp;&nbsp;</span>A single class for both composites and non-composites</a></span></li></ul></li></ul></li></ul></div>

The Composite Pattern is designed to support the uniform treatment of objects in a hierarchy, whether they contain other objects (as part of the hierarchy) or not. Such objects are called composite.

#### Composite Pattern

###### A classic composite/noncomposite hierarchy

In [16]:
# The classic approach is based on having an abstract base class
# for all kinds of composite
import abc


class AbstractItem(metaclass=abc.ABCMeta):

    @abc.abstractproperty
    def composite(self):

        raise NotImplementedError

    def __iter__(self):

        return iter([])
    
    def __str__(self):
        
        raise NotImplementedError

In [17]:
class SimpleItem(AbstractItem):
    """代表了基本部件(noncomposite)的类,
    因为是基本部件, 所以他不需要重载 __iter__ 方法
    """
    
    def __init__(self, name, price=0.0):
        
        self.name = name
        self.price = price
    
    @property
    def composite(self):
        
        return False
    
    def __str__(self):
        
        return '{} : {:.2f}'.format(self.name, self.price)

In [24]:
# 我们没有让 CompositemItem 直接继承 AbstractItem,
# 而是定义一个 AbstractCompositeItem 的中间件
# 他定义了一些组合对象的必须方法: add, remove


class AbstractCompositeItem(AbstractItem):

    def __init__(self, *items):

        self.childern = []
        if items is not None:
            self.add(*items)

    def add(self, first, *items):

        self.childern.append(first)
        if items is not None:
            self.childern.extend(items)

    def remove(self, item):
        self.childern.remove(item)

    def __iter__(self):

        return iter(self.childern)
    
    def __str__(self):
        
        result = ''
        for child in self:
            result += str(child)
            result += '\n'
        return result

In [25]:
# 下面定义 CompositeItem 类


class CompositeItem(AbstractCompositeItem):
    
    def __init__(self, name, *items):
        
        super().__init__(*items)
        self.name = name
    
    @property
    def composite(self):
        
        return True
    
    @property
    def price(self):
        """递归计算各个组件的单价
        """        
        return sum(item.price for item in self)
    
    def __str__(self):
        
        result = '{} : {:.2f}'.format(self.name, self.price)
        result += '\n'
        for child in self:
            result += str(child)
            result += '\n'
        return result

In [26]:
Pencil = SimpleItem('Pencil', 2.5)
Ruler = SimpleItem('Ruler', 3.5)
Eraser = SimpleItem('Eraser', 4.5)

In [27]:
print(Pencil)
print(Ruler)
print(Eraser)

Pencil : 2.50
Ruler : 3.50
Eraser : 4.50


In [28]:
PencilSet = CompositeItem('Pencil Set', SimpleItem('Pencil', 2.5), 
                          SimpleItem('Ruler', 3.5), SimpleItem('Eraser', 4.5))

In [29]:
print(PencilSet)

Pencil Set : 10.50
Pencil : 2.50
Ruler : 3.50
Eraser : 4.50



###### A single class for both composites and non-composites

In [34]:
import itertools


class Item(object):

    def __init__(self, name, *items, price=0.00):

        self.name = name
        self.price = price  # Call setter method
        self.childern = []
        if items is not None:
            self.add(*items)

    @classmethod
    def create(cls, name, price):

        return cls(name, price=price)

    @classmethod
    def compose(cle, name, *items):

        return cls(name, *items)

    @property
    def composite(self):

        return bool(len(self.childern))

    @property
    def price(self):

        if len(self.childern) != 0:
            return sum(item.price for item in self)
        else:
            return self.__price

    @price.setter
    def price(self, price):

        if price < 0:
            raise ValueError('price must be > 0.')
        else:
            self.__price = price

    def add(self, first, *items):

        self.childern.extend(itertools.chain((first, ), items))

    def remove(self, item):

        self.childern.remove(item)

    def __iter__(self):

        return iter(self.childern)

    def __str__(self):

        result = '{} : {:.2f}'.format(self.name, self.price)
        for child in self:
            result += '\n'
            result += str(child)
        return result


def make_item(name, price):

    return Item(name, price=price)

def make_composite(name, *items):
    
    return Item(name, *items)